In [7]:
!pip -q install langchain faiss-cpu langchain-groq  langchain-community chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 7.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 628.3/628.3 kB 41.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 83.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 77.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.2/93.2 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 37.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.6/166.6 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.3/64.3 kB 6.5 MB/s eta 0:00:

In [26]:
from langchain.schema import Document
from langchain.vectorstores import Chroma
from langchain.retrievers import ParentDocumentRetriever
from bs4 import BeautifulSoup
import requests
from langchain_groq import ChatGroq
import re
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.storage import InMemoryStore
from langchain.document_loaders import TextLoader
from langchain.chains import create_retrieval_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_groq import ChatGroq
from langchain.embeddings import HuggingFaceBgeEmbeddings

In [5]:
response = requests.get("https://en.wikipedia.org/wiki/Large_language_model")
soup = BeautifulSoup(response.content, 'html.parser')
text = soup.get_text()
text = re.sub(r"\n+", "\n", text)
with open("text_file.txt", "w",encoding='utf-8') as f:
        f.write(text)
f.close()

loader=TextLoader("text_file.txt", encoding="utf-8")
docs=loader.load()
print("The document is: ")
print(docs[0].page_content[:100])
print("The length is: ")
print(len(docs[0].page_content))

The document is: 

Large language model - Wikipedia
Jump to content
Main menu
Main menu
move to sidebar
hide
		Navigat
The length is: 
111618


## Parent Document Retriever

- Return full docs from smaller chunks look up
- Return bigger chunks for smaller chunks look up

In [3]:


model_name = "BAAI/bge-small-en-v1.5"
encode_kwargs = {'normalize_embeddings': True}

bge_embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs={'device': 'cuda'},
    encode_kwargs=encode_kwargs
)

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

## 1. Retrieving full documents rather than chunks
- we retrieve the full documents.

- This is good to use if the entire documents aren't too big themselves and we aren't going to return many of them

In [17]:

child_splitter = RecursiveCharacterTextSplitter(chunk_size=500)

vectorstore = Chroma(

    embedding_function=bge_embeddings
)

store = InMemoryStore()

full_doc_retriever = ParentDocumentRetriever(
    vectorstore=vectorstore,
    docstore=store,
    child_splitter=child_splitter,
)

In [11]:
full_doc_retriever.add_documents(docs, ids=None)

In [12]:
retrieved_docs = full_doc_retriever.get_relevant_documents("what is the webpage about")

<ipython-input-12-6443e53746da>:1: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retrieved_docs = full_doc_retriever.get_relevant_documents("what is the webpage about")


In [14]:
retrieved_docs[0].page_content.replace("\n"," ")

' Large language model - Wikipedia Jump to content Main menu Main menu move to sidebar hide \t\tNavigation \t Main pageContentsCurrent eventsRandom articleAbout WikipediaContact us \t\tContribute \t HelpLearn to editCommunity portalRecent changesUpload file Search Search Appearance Donate Create account Log in Personal tools Donate Create account Log in \t\tPages for logged out editors learn more ContributionsTalk Contents move to sidebar hide (Top) 1 History 2 Dataset preprocessing Toggle Dataset preprocessing subsection 2.1 Tokenization 2.1.1 BPE 2.1.2 Problems 2.2 Dataset cleaning 2.3 Synthetic data 3 Training and architecture Toggle Training and architecture subsection 3.1 Reinforcement learning from human feedback (RLHF) 3.2 Instruction tuning 3.3 Mixture of experts 3.4 Prompt engineering, attention mechanism, and context window 3.5 Infrastructure 4 Training cost 5 Tool use 6 Agency 7 Compression 8 Multimodality 9 Properties Toggle Properties subsection 9.1 Scaling laws 9.2 Emerge

## Retrieving larger chunks
In many cases, the full documents can be too big. In that case, what we really want to do is to first split the raw documents into larger chunks, and then split it into smaller chunks. We then index the smaller chunks, but on retrieval we retrieve the larger chunks (but still not the full documents).

In [18]:

parent_splitter = RecursiveCharacterTextSplitter(chunk_size=2000)

child_splitter = RecursiveCharacterTextSplitter(chunk_size=400)

vectorstore = Chroma( embedding_function=bge_embeddings)

store = InMemoryStore()

In [19]:
big_chunks_retriever = ParentDocumentRetriever(
    vectorstore=vectorstore,
    docstore=store,
    child_splitter=child_splitter,
    parent_splitter=parent_splitter,
)

In [20]:
big_chunks_retriever.add_documents(docs)

In [21]:
# sub_docs = vectorstore.similarity_search("what is the webpage about")

In [23]:
retrieved_docs = big_chunks_retriever.get_relevant_documents("what is the webpage about")

In [24]:
retrieved_docs[0].page_content

'^ Goodman, Joshua (2001-08-09), A Bit of Progress in Language Modeling, arXiv:cs/0108005, Bibcode:2001cs........8005G\n^ Kilgarriff, Adam; Grefenstette, Gregory (September 2003). "Introduction to the Special Issue on the Web as Corpus". Computational Linguistics. 29 (3): 333–347. doi:10.1162/089120103322711569. ISSN\xa00891-2017.\n^ Banko, Michele; Brill, Eric (2001). "Scaling to very very large corpora for natural language disambiguation". Proceedings of the 39th Annual Meeting on Association for Computational Linguistics - ACL \'01. Morristown, NJ, USA: Association for Computational Linguistics: 26–33. doi:10.3115/1073012.1073017.\n^ Resnik, Philip; Smith, Noah A. (September 2003). "The Web as a Parallel Corpus". Computational Linguistics. 29 (3): 349–380. doi:10.1162/089120103322711578. ISSN\xa00891-2017. Archived from the original on 2024-06-07. Retrieved 2024-06-07.\n^ Halevy, Alon; Norvig, Peter; Pereira, Fernando (March 2009). "The Unreasonable Effectiveness of Data". IEEE Inte

In [27]:
groq="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"
llm=ChatGroq(groq_api_key=groq, model_name="Llama3-8b-8192")
prompt = ChatPromptTemplate.from_template("""
Answer the following question based only on the provided context.
Think step by step before providing a detailed answer. If the answer is not present in the context, simply say 'I don't know the answer' and nothing more.

<context>
{context}
</context>
    <Question>
    {input}
    </Question>""")
question="what is the webpage about"
document_chain = create_stuff_documents_chain(llm, prompt)

retrieval_chain = create_retrieval_chain(big_chunks_retriever, document_chain)

response = retrieval_chain.invoke({"input": question})

output = response["answer"]
output = output.replace("*", "\n")

In [28]:
output

'Based on the provided context, I understand that the webpage is about Artificial Neural Networks, Machine Learning, and Data Mining.'